In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from imageio import imread
from skimage.transform import resize
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
#from keras.models import load_model

% matplotlib inline

In [11]:
image_dir_basepath = '../data/images/'
names = ['3', '29', '35'] # three type of fish
image_size = 224

In [4]:
model_path = '../model/model-resnet50-final.h5' # your fish classifier
model = load_model(model_path)
model._layers.pop(-1) # exclude the dense layer which output the class of fish

# How to pop out the desire layer in Keras
https://github.com/tensorflow/tensorflow/issues/22479

In [15]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [6]:
def load_images(filepaths):
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)
        aligned = resize(img, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
        
    return np.array(aligned_images)

In [13]:
def calc_embs(filepaths, batch_size=1):
    aligned_images = load_images(filepaths)
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))
    
    return embs

In [29]:
def train(dir_basepath, names, max_num_img=10):
    labels = []
    embs = []
    for name in names:
        dirpath = os.path.abspath(dir_basepath + name)
        filepaths = [os.path.join(dirpath, f) for f in os.listdir(dirpath)][:max_num_img]
        _embs = calc_embs(filepaths)
        labels.append([name] * len(_embs))
        embs.append(_embs)
        
    print(len(embs[0]))
    embs = np.concatenate(embs)
    print(embs.shape)
    le = LabelEncoder().fit(labels)
    print(le.shape)
    y = le.transform(labels)
    clf = SVC(kernel='linear', probability=True).fit(embs, y)
    return le, clf

In [9]:
def infer(le, clf, filepaths):
    embs = calc_embs(filepaths)
    pred = le.inverse_transform(clf.predict(embs))
    return pred

In [30]:
le, clf = train(image_dir_basepath, names)

3
(9, 41)


ValueError: bad input shape (3, 3)

In [ ]:
test_dirpath = os.path.join(image_dir_basepath, 'Test')
test_filepaths = [os.path.join(test_dirpath f) for f in os.listdir(test_dirpath)]

In [ ]:
pred = infer(le, clf, test_filepaths)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

for i in range(3):
    axes[i].set_title('Prediction :' + str(pred[i]))
    axes[i].imshow(imread(test_filepaths[i]))
    axes[i].set_xticks([])
    axes[i].set_yticks([])
plt.show()